In [60]:
using CSV
using DataFrames
using Revise
using StatsBase
using StochasticPrograms
using Gurobi
using JuMP
using StatsPlots
using Distributions
#cd("/Users/frankiecho/Library/CloudStorage/OneDrive-TheUniversityofQueensland/Documents/GitHub/koala-uncertainty/")
cd(raw"C:\Users\uqfcho\OneDrive - The University of Queensland\Documents\GitHub\koala-uncertainty")

include("optim-functions.jl")

fcn_evaluate_solution (generic function with 1 method)

In [61]:
cost_df = CSV.read("data/spatial_predictions_10yr.csv", DataFrame);
kitl_index_full = CSV.read("data/kitl_prop_climate.csv", DataFrame);
stratified_samples = CSV.read("data/stratified_sample.csv", DataFrame);
climateProjList = ["CCCMA_R1", "CCCMA_R2", "CCCMA_R3", "CSIRO_R1", "CSIRO_R2", "CSIRO_R3", "ECHAM_R1", "ECHAM_R2", "ECHAM_R3", "MIROC_R1", "MIROC_R2", "MIROC_R3"];

Calculate opportunity costs given an inflation rate

In [62]:
inflation_rate = 0.02;
t = 1:60;

function cost_to_ts(cost, area, adopt, idx_before=1:3, idx_after=4:6, inflation_rate = 0.02)
  tp = 1:6 # time periods
  t_blocks = [(1:10) .+ (td-1)*10 for td in tp];
  delta = (1+inflation_rate).^(t);
  cost_ts = (cost .* 1000 .* area .* adopt ./ 10); # Cost per-year
  cost_ts_full = [cost_ts .* d for d in delta];
  cost_delta = mapreduce(permutedims, vcat, cost_ts_full)';
  cost_blocks = mapreduce(permutedims, vcat, [sum(cost_delta[:,td], dims = 2) for td in t_blocks])';
  cost_before = sum(cost_blocks[:, idx_before], dims=2); # Cost start at 2020
  cost_after = sum(cost_blocks[:, idx_after], dims=2);
  return(cost_before, cost_after)
end

adoption = innerjoin(cost_df[:, [:NewPropID, :MeanWTA, :SDWTA, :MeanAdopt, :SDAdopt, :MeanProp, :SDProp, :AREA]], kitl_index_full[cmp.(kitl_index_full.climate_model,"Avg").==0, [:NewPropID]], on = :NewPropID);
cost_before, cost_after = cost_to_ts(adoption.MeanWTA, adoption.AREA, adoption.MeanAdopt)

([25815.051874207988; 19811.55790732091; … ; 10868.211392272427; 10733.907768284918;;], [46760.393256575655; 35885.894914560144; … ; 19686.26060387671; 19442.988178779877;;])

In [63]:
kitl_threshold = 0.25; # cut-off for habitats deemed "high-quality"
ii = 1; # Stratified sample to use
kitl_index = innerjoin(kitl_index_full, adoption[:, [:NewPropID, :AREA, :MeanProp, :MeanAdopt]], on = :NewPropID);

In [64]:

t_symbols = [:t0, :t1, :t2, :t3, :t4, :t5, :t6, :t7]
tt = 4 # Time period when uncertainty is revealed

4

In [65]:
kitl_threshold_values = (kitl_index[:, t_symbols] .+ 0.5) .* kitl_index.AREA .* kitl_index.MeanProp;

Subset dataset to only stratified sampled cells

In [66]:
stratified_df = DataFrame(NewPropID = vec(stratified_samples[:, ii]))
cost_inputs = DataFrame(NewPropID = adoption.NewPropID, cost_before = vec(cost_before), cost_after = vec(cost_after));
cost_subset = innerjoin(stratified_df, cost_inputs, on = :NewPropID);
metric_inputs = DataFrame(NewPropID = kitl_index.NewPropID, climate_model = kitl_index.climate_model);
metric_inputs = hcat(metric_inputs, kitl_threshold_values);
metric_subset = innerjoin(stratified_df, metric_inputs, on = :NewPropID);
filter!(row -> cmp(row.climate_model, "Avg") != 0, metric_subset);
filter!(row -> row.NewPropID ∈ cost_subset.NewPropID, metric_subset);
filter!(row -> row.NewPropID ∈ metric_subset.NewPropID, cost_subset);

In [67]:
climateProjList = ["CCCMA_R1", "CCCMA_R2", "CCCMA_R3", "CSIRO_R1", "CSIRO_R2", "CSIRO_R3", "ECHAM_R1", "ECHAM_R2", "ECHAM_R3", "MIROC_R1", "MIROC_R2", "MIROC_R3"];
metric_subset

,NewPropID,climate_model,t0,t1,t2,t3,t4,t5
,Int64,String15,Float64,Float64,Float64,Float64,Float64,Float64
1,3039,CCCMA_R1,0.952858,0.751328,0.719133,0.671101,0.666848,0.662095
2,3039,CCCMA_R2,0.952868,0.740683,0.697315,0.636743,0.635783,0.637545
3,3039,CCCMA_R3,0.952864,0.74244,0.698499,0.631515,0.632762,0.635284
4,3039,CSIRO_R1,0.952946,0.752189,0.727038,0.699278,0.694496,0.684893
5,3039,CSIRO_R2,0.952938,0.747661,0.717347,0.682489,0.672919,0.664046
6,3039,CSIRO_R3,0.952742,0.735542,0.692134,0.642075,0.656035,0.668263
7,3039,ECHAM_R1,0.952658,0.739759,0.698253,0.647633,0.663895,0.502897
8,3039,ECHAM_R2,0.952495,0.73289,0.683032,0.62385,0.643952,0.660906
9,3039,ECHAM_R3,0.952872,0.740993,0.700543,0.651823,0.657173,0.662571


In [68]:
N = length(unique(metric_subset.NewPropID))
S = length(climateProjList)
M₁ = zeros(N, length(1:(tt-1)), S)
M₂ = zeros(N, length(tt:length(t_symbols)), S)
for t=1:length(t_symbols)
  for s=1:S
    metric_unstacked = unstack(metric_subset, :NewPropID, :climate_model, t_symbols[t])
    if (t < tt)
      M₁[:,t,:] = Matrix(metric_unstacked[:, climateProjList])
    else
      M₂[:,t-tt+1,:] = Matrix(metric_unstacked[:, climateProjList])
    end
  end
end

In [69]:
r = Realisation(cost_subset.cost_before, cost_subset.cost_after, M₁, M₂)

Realisation([12432.384199054242, 9874.475325585636, 12056.32175033267, 133523.5588664438, 140181.62088322485, 172174.98385832383, 173319.1040696074, 150586.13521790563, 98575.10522015524, 18890.70256603912  …  138562.5829890136, 781776.5238447646, 340520.8282488327, 578458.7222795164, 57029.356948896595, 1.6321440280791444e6, 4.636846715301958e6, 42641.481241635, 165754.5185489979, 11764.13696684975], [22519.543136980406, 17886.245267942286, 21838.358064142318, 241859.4451034392, 253919.60286521405, 311871.1515045831, 313943.5669028983, 272765.94043230754, 178555.1587447353, 34217.892924845924  …  250986.93982043222, 1.4160799626462667e6, 616806.346876992, 1.0477979075266273e6, 103300.786343349, 2.9564029921862693e6, 8.39900601147379e6, 77239.14101036148, 300241.3672912018, 21309.105771881797], [0.9528583023712631 0.7513279267461038 0.7191326699118952; 0.906112065442047 0.5767276008118271 0.5514838844466012; … ; 22.647864163716072 10.968161164922671 10.968161164922671; 1.53636334421944

In [70]:
[sum.(eachcol(M₂[:,:,s])) for s=1:S]

12-element Vector{Vector{Float64}}:
 [287385.3971800607, 278266.3003196805, 269619.59996243706, 262059.35087797587, 255985.44202846126]
 [279578.0825761811, 271570.48017885897, 263998.0891830905, 257320.40535914357, 251833.18617252033]
 [283207.91745669086, 277523.0282164429, 271981.8527398872, 266919.0888236627, 262626.22682727]
 [290409.3395039771, 284905.59255746694, 279184.5140362365, 273859.50279106724, 269051.6794065286]
 [292859.59337934316, 286980.5484465358, 281346.2657746483, 276202.4192896288, 271673.29956002586]
 [289584.0746719709, 285529.50941096677, 281471.0864096532, 277582.5211830083, 273974.06225072924]
 [290179.59314169135, 280205.08648061723, 240710.08001171917, 238841.19772634064, 257009.67226389094]
 [290330.0521879771, 282772.4380735157, 275472.6117483712, 269030.5673790496, 263887.9993910194]
 [285212.48577986285, 277950.36105164106, 271368.7259310348, 265972.69994169223, 261986.42633442488]
 [290464.2492639938, 281302.35578066134, 272730.6699117871, 265566.0468

In [71]:
solution_no_recourse = fcn_two_stage_opt_saa([r], add_recourse = false, terminate_recourse = false)
solution_add_recourse = fcn_two_stage_opt_saa([r], add_recourse = true, terminate_recourse = false)
solution_terminate_recourse = fcn_two_stage_opt_saa([r], add_recourse = false, terminate_recourse = true)
solution_full_recourse = fcn_two_stage_opt_saa([r], add_recourse = true, terminate_recourse = true)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 228985 rows, 238426 columns and 2756194 nonzeros
Model fingerprint: 0x97e77682
Coefficient statistics:
  Matrix range     [3e-01, 2e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 228889 rows and 228889 columns
Presolve time: 0.50s
Presolved: 96 rows, 9537 columns, 915552 nonzeros

Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 4.560e+03
 Factor NZ  : 4.656e+03 (roughly 4 MB of memory)
 Factor Ops : 2.995e+05 (less than 1 second per iteration)
 Threads    : 4

 

(model = A JuMP Model
Minimization problem with:
Variables: 238426
Objective function type: VariableRef
`AffExpr`-in-`MathOptInterface.GreaterThan{Float64}`: 114541 constraints
`AffExpr`-in-`MathOptInterface.LessThan{Float64}`: 114444 constraints
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 238425 constraints
`VariableRef`-in-`MathOptInterface.LessThan{Float64}`: 238425 constraints
Model mode: AUTOMATIC
CachingOptimizer state: ATTACHED_OPTIMIZER
Solver name: Gurobi
Names registered in the model: w, x, y, z, obj_value = 8.823264558816351e7, x = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0], y = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0], w = [0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 0.0; 0.0 0.0 … 0.0 0.0])

In [72]:
objective_value(solution_no_recourse.model)


1.3069782869306749e8

In [73]:
objective_value(solution_full_recourse.model)

8.823264558816351e7

In [74]:
objective_value(solution_add_recourse.model)

9.079764309146556e7

In [75]:

sum(value.(solution_full_recourse.model[:x]) .- value.(solution_full_recourse.model[:w])[:,10])

36.07878279002094

In [76]:
r.C₁' * value.(solution_full_recourse.model[:x])

3.0161628092569586e7

In [77]:
cost_nr, metric_nr = fcn_evaluate_solution(solution_no_recourse.model, [r])
cost_add, metric_add = fcn_evaluate_solution(solution_add_recourse.model, [r])
cost_terminate, metric_terminate = fcn_evaluate_solution(solution_terminate_recourse.model, [r])
cost_full, metric_full = fcn_evaluate_solution(solution_full_recourse.model, [r])

([8.668415448786359e7; 9.06981694978467e7; … ; 8.509157454779565e7; 8.453114827258633e7;;], [7655.322715887657 7143.731230940931 … 7040.5785490445105 7000.0; 7655.192470887142 7118.662906011406 … 7084.480090789831 6999.999999999998; … ; 7656.247907803548 7192.236456120176 … 7021.345583713232 7000.000000000001; 7656.54193053339 7174.495429383417 … 7024.499567472478 7000.0;;;])

In [78]:
CSV.write("data/habitat_size_add_recourse.csv", DataFrame(metric_add[:,:,1]', climateProjList))
CSV.write("data/habitat_size_no_recourse.csv", DataFrame(metric_nr[:,:,1]', climateProjList))
CSV.write("data/habitat_size_terminate_recourse.csv", DataFrame(metric_terminate[:,:,1]', climateProjList))
CSV.write("data/habitat_size_full_recourse.csv", DataFrame(metric_full[:,:,1]', climateProjList))

CSV.write("data/cost_add_recourse.csv", DataFrame(cost_add', climateProjList))
CSV.write("data/cost_no_recourse.csv", DataFrame(cost_nr', climateProjList))
CSV.write("data/cost_terminate_recourse.csv", DataFrame(cost_terminate', climateProjList))
CSV.write("data/cost_full_recourse.csv", DataFrame(cost_full', climateProjList))

"data/cost_full_recourse.csv"

In [79]:
objective_value(solution_full_recourse.model)

8.823264558816351e7

In [80]:
function fcn_tidy_two_stage_solution(solution, prop_id, climate_models)
  out = hcat(prop_id, solution.x, solution.y, solution.w)
  y_array = ["y_" * c for c in climate_models];
  w_array = ["w_" * c for c in climate_models];
  colnames = vcat(["NewPropID"; "x"], y_array, w_array);
  return DataFrame(out, colnames);
end

function fcn_tidy_two_stage_solution_sum(solution, prop_id)
  out = hcat(prop_id, solution.x, sum(solution.y, dims=2), sum(solution.w, dims=2))
  colnames = vcat(["NewPropID"; "x"; "sum_y"; "sum_w"]);
  return DataFrame(out, colnames);
end

climate_models = climateProjList;
solution_df = map((c->fcn_tidy_two_stage_solution_sum(c, cost_subset.NewPropID)), [solution_no_recourse, solution_add_recourse, solution_terminate_recourse, solution_full_recourse]);

In [81]:
CSV.write("data/solution_no_recourse_kitl.csv", solution_df[1]);
CSV.write("data/solution_add_only_kitl.csv", solution_df[2]);
CSV.write("data/solution_terminate_only_kitl.csv", solution_df[3]);
CSV.write("data/solution_full_recourse_kitl.csv", solution_df[4]);

## Evaluate solution on sample
* Variability in WTA
* Adoption
* Proportion

In [82]:
adoption_subset = innerjoin(stratified_df, adoption, on = :NewPropID);
nsims = 100;
adopt_binary_sim = [(rand.(Normal.(adoption_subset.MeanAdopt,adoption_subset.SDAdopt))) .> rand(nrow(adoption_subset)) for ns in 1:nsims];
prop_sim = [(rand.(Normal.(adoption_subset.MeanProp,adoption_subset.SDProp)))  for ns in 1:nsims];
cost_sim = [(rand.(Normal.(adoption_subset.MeanWTA, adoption_subset.SDWTA))) for ns in 1:nsims];

In [83]:
cost_before_sim = [adopt_binary_sim[ns] .* cost_to_ts(cost_sim[ns], adoption_subset.AREA, prop_sim[ns])[1] for ns in 1:nsims]
cost_after_sim = [adopt_binary_sim[ns] .* cost_to_ts(cost_sim[ns], adoption_subset.AREA, prop_sim[ns])[2] for ns in 1:nsims]

100-element Vector{Matrix{Float64}}:
 [12138.784333992418; 9600.467094351792; … ; 167056.79658540926; 14955.895050722185;;]
 [11154.429349139515; 9128.353857047314; … ; 174142.29981401752; 14781.485060055908;;]
 [13666.195534968518; 9152.70908954963; … ; 0.0; 0.0;;]
 [9642.972805422605; 7600.67329309965; … ; 182094.16769293178; 11230.590049969058;;]
 [8059.6004660395665; 7778.145905629642; … ; 201148.35276232834; 16058.82629114792;;]
 [9805.967955778466; 8080.8413828106; … ; 179432.0643910075; 11771.0616729242;;]
 [7864.61993320615; 5670.320217552855; … ; 199733.54892694467; 12595.707087593955;;]
 [0.0; 9430.779624579656; … ; 208963.37002816703; 14584.074774609835;;]
 [10459.484324265595; 9191.976985219993; … ; 0.0; 10869.215663653933;;]
 [11779.327774863217; 0.0; … ; 0.0; 12825.220336118113;;]
 ⋮
 [0.0; 0.0; … ; 174203.5405993745; 9139.60096678234;;]
 [9592.01164466617; 7922.741735131331; … ; 198737.36590894795; 23024.78851861837;;]
 [11593.910213060506; 8942.276367032171; … ; 164765.

In [102]:
@define_scenario BehaviourScenario = begin
    C₁::Array{Float64,2}
    C₂::Array{Float64,2}
    M₁::Array{Float64,2}
    M₂::Array{Float64,2}
    @zero begin
        return BehaviourScenario(0.0, 0.0, 0.0, 0.0)
    end
end

@sampler RealisationSampler = begin
    C₁::Array{Float64,2}
    C₂::Array{Float64,2}
    M₁::Array{Float64,2}
    M₂::Array{Float64,2}

    RealisationSampler(Adopt::Array{Float64,2}, Prop::Array{Float64,2}, WTA::Array{Float64,2}, Area::Array{Float64,1}) = new(C₁, C₂, M₁, M₂)
    @sample BehaviourScenario begin

        MeanAdopt = sampler.Adopt[:,1]
        SDAdopt = sampler.Adopt[:,2]
        MeanProp = sampler.Prop[:,1]
        SDProp = sampler.Prop[:,2]
        MeanWTA = sampler.WTA[:,1]
        SDWTA = sampler.WTA[:,2]
        adopt_binary = (rand.(Normal.(MeanAdopt,SDAdopt))) .> rand(nrow(adoption_subset))
        sample_C₁ = cost_to_ts(MeanWTA + SDWTA .* randn(size(SDWTA)), sampler.Area, MeanProp + SDProp .* randn(size(SDProp)))[1]
        sample_C₂ = cost_to_ts(MeanWTA + SDWTA .* randn(size(SDWTA)), sampler.Area, MeanProp + SDProp .* randn(size(SDProp)))[2]
        
        sample_M₁ = M₁ .* adopt_binary
        sample_M₂ = M₂ .* adopt_binary
        return BehaviourScenario(sample_C₁, sample_C₂, sample_M₁, sample_M₂)
    end
end

ErrorException: syntax: unexpected "end"

In [94]:
M₁

9537×3×12 Array{Float64, 3}:
[:, :, 1] =
   0.952858    0.751328    0.719133
   0.906112    0.576728    0.551484
   0.965209    0.622461    0.603473
  19.3438     10.5472     10.3884
  17.9918     10.6718     10.3888
  17.2382     10.1043     10.015
  31.691      25.0376     24.5274
  27.1967     22.2697     21.1116
  12.9219      6.42881     6.39403
   1.54966     0.779297    0.779297
   ⋮                     
  99.7447     59.1988     59.0462
  26.8528     26.3019     25.6263
  38.721      36.3273     35.8318
  15.6515     14.874      14.7993
  87.4083     87.3876     87.3656
 863.655     695.258     684.592
   1.77822     1.77822     1.77822
  22.6479     10.9682     10.9682
   1.53636     1.36189     1.26003

[:, :, 2] =
   0.952868    0.740683    0.697315
   0.906094    0.569935    0.541646
   0.965135    0.609632    0.577852
  19.3438     10.4573     10.1969
  17.9919     10.542      10.0996
  17.238      10.0203      9.83556
  31.6913     24.8835     24.3516
  27.197      21.868

In [93]:
M₁ .* adopt_binary_sim[1]

9537×3×12 Array{Float64, 3}:
[:, :, 1] =
   0.952858    0.751328    0.719133
   0.906112    0.576728    0.551484
   0.0         0.0         0.0
  19.3438     10.5472     10.3884
  17.9918     10.6718     10.3888
  17.2382     10.1043     10.015
  31.691      25.0376     24.5274
   0.0         0.0         0.0
   0.0         0.0         0.0
   1.54966     0.779297    0.779297
   ⋮                     
  99.7447     59.1988     59.0462
  26.8528     26.3019     25.6263
  38.721      36.3273     35.8318
   0.0         0.0         0.0
   0.0         0.0         0.0
 863.655     695.258     684.592
   1.77822     1.77822     1.77822
  22.6479     10.9682     10.9682
   1.53636     1.36189     1.26003

[:, :, 2] =
   0.952868    0.740683    0.697315
   0.906094    0.569935    0.541646
   0.0         0.0         0.0
  19.3438     10.4573     10.1969
  17.9919     10.542      10.0996
  17.238      10.0203      9.83556
  31.6913     24.8835     24.3516
   0.0         0.0         0.0
   0.0      

In [84]:
realisation_samples = [Realisation(cost_before_sim[ns], cost_after_sim[ns], cat([M₁[:,:,s] .* adopt_binary_sim[ns] for s=1:S]..., dims = 3), cat([M₂[:,:,s] .* adopt_binary_sim[ns] for s=1:S]..., dims = 3)) for ns=1:nsims];

In [85]:
cost_mat, metric_mat = fcn_evaluate_solution(solution_add_recourse.model, realisation_samples)


([4.836035809653333e7 4.936171104900645e7 … 5.377926407078411e7 5.5347633197771475e7; 5.276888929732552e7 5.1988243898918405e7 … 5.6304275353330284e7 5.893939381002958e7; … ; 4.754384688309372e7 4.8472284987824455e7 … 5.278874670235933e7 5.4480091399424054e7; 4.722562497295572e7 4.8147295159348086e7 … 5.2392884644485265e7 5.413135503147497e7], [7359.015642108733 6600.308990655611 … 6462.623214765344 6364.741097216612; 7358.169903762256 6524.904165354419 … 6545.874510671387 6411.848919432418; … ; 7359.984870555047 6613.381691260312 … 6450.831151256737 6377.275285499973; 7360.4306912858565 6613.951951187207 … 6493.874040754174 6452.224061431858;;; 7688.455159382069 6899.098094934772 … 6771.621112200021 6669.482141956936; 7687.535480809726 6819.297264526559 … 6822.920524731018 6679.267240611163; … ; 7689.391732938817 6908.020117167567 … 6731.849708431375 6658.494209796457; 7689.944205340113 6912.777486601047 … 6789.933275900508 6747.9051389865845;;; 7715.5952594305045 6938.0827137151655 …

In [86]:
solution_no_recourse_saa = fcn_two_stage_opt_saa(realisation_samples[1:50], add_recourse = false, terminate_recourse = false)
solution_add_recourse_saa = fcn_two_stage_opt_saa(realisation_samples[1:50], add_recourse = true, terminate_recourse = false)

Set parameter Username
Academic license - for non-commercial use only - expires 2023-12-16
Gurobi Optimizer version 9.5.2 build v9.5.2rc0 (win64)
Thread count: 6 physical cores, 6 logical processors, using up to 6 threads
Optimize a model with 233738 rows, 238426 columns and 98087890 nonzeros
Model fingerprint: 0x7f488f98
Coefficient statistics:
  Matrix range     [3e-01, 1e+08]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 7e+03]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.

Concurrent LP optimizer: primal simplex, dual simplex, and barrier
Showing barrier log only...

Presolve removed 0 rows and 0 columns (presolve time = 53s) ...
Presolve removed 228888 rows and 228888 columns (presolve time = 57s) ...
Presolve removed 228888 rows and 228888 columns (presolve time = 60s) ...
Presolve removed 228888 rows and 228888 columns (presolve time = 75s) ...
Presolve removed 228888 r

MathOptInterface.ResultIndexBoundsError{MathOptInterface.ObjectiveValue}: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.